# SPIE-AAPM Lung CT Challenge

Zadaniem dla tego zbioru jest klasyfikacja zmian w płucach, na podstawie zdjęć z tomografu komputerowego.

Na początku wczytane zostaną pliki excel z informacjami na temat lokalizacji i rodzaju zaobserwowanych zmian w płucach.
Są dwa pliki: pierwszy z nich ("CalibrationSet") jest z założenia zbiorem treningowym, drugi ("TestSet") jest zbiorem testowym.

### Dane dodatkowe tabelaryczne

In [32]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
train_set = pd.read_excel("CalibrationSet_NoduleData.xlsx", nrows=10)
test_set = pd.read_excel("TestSet_NoduleData_PublicRelease_wTruth.xlsx", nrows=73)

In [34]:
train_set

,Scan Number,"Nodule Center x,y Position*",Nodule Center Image,Diagnosis
0,CT-Training-lc001,"120, 325",135,malignant
1,CT-Training-lc002,"139, 359",70,malignant
2,CT-Training-lc003,"375, 323",70,malignant
3,CT-Training-lc008,"95, 328",65,malignant
4,CT-Training-lc009,"145, 299",63,malignant
5,CT-Training-be001,"405, 296",169,benign
6,CT-Training-be002,"184, 268",117,benign
7,CT-Training-be006,"449, 266",241,benign
8,CT-Training-be007,"385, 206",194,benign
9,CT-Training-be010,"120, 336",69,benign


In [35]:
test_set

,Scan Number,Nodule Number,"Nodule Center x,y Position*",Nodule Center Image,Final Diagnosis
0,LUNGx-CT001,1,"135, 303",142,Benign nodule
1,LUNGx-CT002,1,"330, 348",205,Benign nodule
2,LUNGx-CT002,2,"364, 212",150,Benign nodule
3,LUNGx-CT003,1,"374, 374",146,Primary lung cancer
4,LUNGx-CT004,1,"197, 290",41,Primary lung cancer
...,...,...,...,...,...
68,LUNGx-CT057,1,"396, 353",169,Benign nodule
69,LUNGx-CT058,1,"377, 234",157,Benign nodule
70,LUNGx-CT059,1,"75, 288",235,Benign nodule
71,LUNGx-CT059,2,"371, 207",258,Benign nodule


Pierwsze co rzuca się w oczy to fakt, że tabelka train i test różnią się od siebie. W dalszej części pracy konieczne będzie ujednolicenie obu tabel.

Oprócz tego mamy do czynienia z istotnie zaburzoną proporcją ilości danych testowych do danych treningowych, danych testowych jest ponad 7 razy więcej. Prawdopodobnie w przyszłości połącze te tabele i dokonam własnego podziału na zbiór testowy i treningowy.

Kolumna mówiąca o pozycji zmiany również będzie wymagała szlifowania. Aktualnie współrzędne zmiany figurują jako string współrzędnych x i y oddzielonych przecinkiem. Jeśli zmienna ta ma być wykorzystana w przyszłości będzie wymagała podziału na dwie osobne zmienne w formacie int.


In [52]:
set(test_set["Final Diagnosis"])

{'Benign nodule',
 'Primary lung cancer',
 'Primary lung cancer ',
 'Suspicious malignant nodule'}

Jak widać, kolumna Final Diagnosis zawiera 4 różne stringi, podczas gdy spodziewaną liczbą klas było 2. Klasa "Primary lung cancer" prawdopodobnie w wyniku błędu reprezentowana jest przez dwa różne stringi różniące się jedynie spacją na końcu. Klasa "Nodule" została dodatkowo podzielona na benign (łagodny, niegroźny) i suspicious (podejrzany).

In [56]:
print("Benign nodule count:")
print(sum(test_set["Final Diagnosis"] == "Benign nodule"))
print("Primary lung cancer count:") 
print(sum(test_set["Final Diagnosis"] == "Primary lung cancer") + sum(test_set["Final Diagnosis"] == "Primary lung cancer "))
print("Suspicious malignant nodule count:")
print(sum(test_set["Final Diagnosis"] == "Suspicious malignant nodule"))

Benign nodule count:
37
Primary lung cancer count:
34
Suspicious malignant nodule count:
2


Mamy więc do czynienia ze zbiorem zrównoważonym (zarówno w przypadku test jak i train), w którym mamy bardzo zbliżoną liczbę zmian poważnych jak i łagodnych.

### Dane obrazkowe

Na potrzeby przykładu weźmy pod uwagę skan numer LUNGx-CT002. Skan ten zawiera dwie zmiany: na 205-tym zdjęciu i na 150-tym.
Do podglądu plików w formacie DICOM użyta została aplikacja "MicroDicom".

#### Zmiany łagodne

![](CT002-150.png)

Powyżej widać zdjęcie 150 ze scanu CT002. Czerwoną strzałką zaznaczyłem pozycję X = 364, Y = 212 zgodnie z danymi z tabeli testowej. Wyraźnie widać tam jakąś zmianę. Jako, że nie mam wiedzy medycznej, nie jestem w stanie stwierdzić czym zaznaczony punkt różni się od dwóch pozostałych na lewo od niego, dla mnie wyglądają one identycznie i obawiam się, że model też może mieć problemy z wykryciem tych zmian.

![](CT002-205.png)

Druga zmiana u tego samego pacjęta wyróżnia się już zdecydowanie bardziej, jest dużo większa.
Obie powyższe zmiany są przykładem zmian łągodnych.

#### Zmiany poważne

![](CT003-146.png)

Zmiana zaklasyfikowana przez lekarzy jako "Primary lung cancer" jest widocznie bardziej nieregularna w kszatałcie i poszarpana.

#### Artefakt

![](artefakt.png)

Podczas przeglądania zbioru danych natknąłem się na nietypowy tomograf. Na lewej piersi pacjenta widzimy bardzo jasny (aż zakłucający resztę obrazu), nienaturalny obiekt. Nie jestem pewny co to jest, podejrzewam, że może to być rozrusznik serca (umiejscowaienie się zgadza), albo jakiś przedmiot, którego pacjent zapomniał wyjąć z piersiowej kieszeni.
W każdym razie, tego typu anomalia może stanowić wyzwanie dla algorytmu identyfikującego zmiany.

### Możliwe problemy

Po pierwsze, tomografy zostały wykonane w różny sposób: niektóre znich oznaczone na zdjęciach jako "CT INFUSED" wykorzystują specjalny barwnik poprawiający kontrast zdjęcia, inne oznaczone jako "CT NON-INFUSED" wykonane zostały w standardowy sposób.
Prawdę powiedziawszy, nie jestem w stanie wskazać różnicy pomiędzy zdjęciami wykonanymi na dwa różne sposoby, ale niewykluczone, że algorytm będzie w stanie zauważyć różnicę i niepoprawnie sugerować się tym faktem.

Kolejnym czynnikiem który może niekorzystnie wpłynąć na klasyfikację guzów są adnotacje na lewo i prawo od zdjęcia. Mamy tam np informację o numerze pacjenta, płci czy ustawieniach tomografu. Algorytm może znaleźć niepoprawną zależność i predykować na ich podstawie zamiast na podstawie zdjęć.

Rozwiązać będzie trzeba również problem skupienia uwagi algorytmu. Niezbędne będzie przy uczeniu "pokazanie algorytmowi gdzie ma patrzeć". Będzie to utrudnione z powodu sposobu w jaki zostały zlokalizowane te zmiany: mamy współrzędne zmiany tylko na jednym z wielu obejmujących ją zdjęć.

# RIDER Lung CT


W tym zbiorze danych mamy po dwa tomografy na pacjenta, zadaniem dla algorytmu będzie stwierdzenie czy podane dwa tomografy przedstawiają tego samego pacjenta.

Zdjęcia zostały wykonane na tym samym sprzęcie, a drugie badanie tomografem dla danego pacjenta następowało po maksymalnie 15 minutach od pierwszego.

Dane tabelaryczne dla tego zadania składają się z lokalizacji zmiany w płucach stwierdzonych na podstawie pierwszego i drugiego skanu. Dane te nie będą potrzebne w moim zadaniu, dlatego pomijam je.

![](dane.png)

Najistotniejszym problemem tego zbioru wobec przedstawionego zadania są metadane zawarte na zdjęciach. Możemy tu odczytać informacje takie jak data skanu, data urodzenia czy numer pacjenta. Oczywiście przy pomocy tych danych można jednoznacznie stwierdzić czy dwa różne tomografy przedstawiają tego samego pacjenta, jednak nie to jest celem zadania. Dane te należy więc napewno usunąć.

Równie istotnym problemem jest ilość "przekrojów" dla danego pacjenta objawiająca się ilością zdjęć. Okazuje się, że ilość zdjęć jakie wykonuje tomograf jest identyczna dla tego samego pacjenta, ale różni się pomiędzy różnymi pacjentami. Bez problemu można zidentyfikować tomografy przedstawiające tego samego pacjenta na podstawie ilości "sliceów" na które podzielony został skan. Aby temu zaradzić, można usunąć część początkowych i końcowych sliceów z każdego tomografu, tak aby wszystkie skany zawierały tyle samo zdjęć.